In [4]:
import scipy.stats as stats
import pandas as pd
import math


In [5]:
#import sample df;
sampleData = pd.read_csv('sampleData.csv');

In [6]:
sampleData

,Study Name,PMID,a,b,c,d
0,myStudy,1111,3,18,6,17
1,myStudy2,2222,1,34,8,6


In [7]:
#add information!!

#find total number of participants
only2x2TableData = sampleData[['a', 'b', 'c', 'd']];
sumAcrossRows = only2x2TableData.sum(axis = 1);
sampleData['totalPop'] = sumAcrossRows;

sampleData

,Study Name,PMID,a,b,c,d,totalPop
0,myStudy,1111,3,18,6,17,44
1,myStudy2,2222,1,34,8,6,49


In [8]:
#given abcd values, what is the p-value using fishers test?
def myFishersTestFunction(abcdDataRow):
    indivTable = abcdDataRow.values
    organizedTable = [
        [indivTable[0], indivTable[1]],
        [indivTable[2], indivTable[3]]
    ]
    [oddsRatio, pValue] = stats.fisher_exact(organizedTable)

    return pValue


In [9]:
#determine p-value from Fisher's Exact Test
only2x2TableData = sampleData[['a', 'b', 'c', 'd']];

myPValuesOriginal = []

for index, rows in only2x2TableData.iterrows():
    pValue = myFishersTestFunction(only2x2TableData.iloc[index])
    myPValuesOriginal.append(pValue)
    
sampleData['pValuesOriginal'] = myPValuesOriginal

In [10]:
sampleData

,Study Name,PMID,a,b,c,d,totalPop,pValuesOriginal
0,myStudy,1111,3,18,6,17,44,0.461753
1,myStudy2,2222,1,34,8,6,49,0.000052


In [13]:
#find the fragility index and fragility quotient
only2x2TableData = sampleData[['a', 'b', 'c', 'd']];

myFIValues = []
newPValues = []

for index, rows in only2x2TableData.iterrows():
    currentStudy = only2x2TableData.iloc[index]
    pValue = myFishersTestFunction(currentStudy)
    
    counter = 0
    # is the pValue insig?
    while (pValue < 0.05):
        currentStudy['a'] = currentStudy['a'] + 1
        currentStudy['b'] = currentStudy['b'] - 1
        counter = counter + 1
        pValue = myFishersTestFunction(currentStudy)
        
    myFIValues.append(counter)
    newPValues.append(pValue)

In [15]:
myFIValues

[0, 8]

In [21]:
#add these new FI and p values to main table
sampleData['FI'] = myFIValues;
sampleData['FQ'] = sampleData['FI'] / sampleData['totalPop']
sampleData['newPValue'] = newPValues;

In [22]:
sampleData

,Study Name,PMID,a,b,c,d,totalPop,pValuesOriginal,FI,newPValue,FQ
0,myStudy,1111,3,18,6,17,44,0.461753,0,0.461753,0.000000
1,myStudy2,2222,1,34,8,6,49,0.000052,8,0.050449,0.163265
